In [54]:
import pandas as pd
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [167]:
base = pd.read_csv('games.csv')

In [168]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [169]:
base = base.drop('Name', axis = 1)

In [170]:
base

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,76.0,51.0,8,322.0,E
1,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,82.0,73.0,8.3,709.0,E
3,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,80.0,73.0,8,192.0,E
4,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16714,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16715,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN
16716,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16717,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [171]:
categorical = [
    "Platform",
    "Genre",
    "Publisher",
    "Rating"
]

In [172]:
import json

In [173]:
import numpy as np

In [174]:
def categorical_to_number(data, variables):
    for v in variables:
        values = list(set(data[v]))
        values = [x for x in values if pd.notna(x)]
        replace = {value: idx for idx, value in enumerate(values)}
        data[v].replace(replace, inplace = True)
        with open(f'{v}.json','w') as file:
            json.dump(replace, file, indent = 4)
    return data
        

In [175]:
base = categorical_to_number(base, categorical)

In [176]:
base

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,18,2006.0,2.0,576.0,41.36,28.96,3.77,76.0,51.0,8,322.0,7.0
1,4,1985.0,7.0,576.0,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,18,2008.0,9.0,576.0,15.68,12.76,3.79,82.0,73.0,8.3,709.0,7.0
3,18,2009.0,2.0,576.0,15.61,10.93,3.28,80.0,73.0,8,192.0,7.0
4,22,1996.0,11.0,576.0,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16714,15,2016.0,0.0,293.0,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16715,0,2006.0,2.0,258.0,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN
16716,10,2016.0,3.0,259.0,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16717,2,2003.0,7.0,143.0,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [177]:
base = base[base['User_Score'] != "tbd"]
base

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,18,2006.0,2.0,576.0,41.36,28.96,3.77,76.0,51.0,8,322.0,7.0
1,4,1985.0,7.0,576.0,29.08,3.58,6.81,NaN,NaN,NaN,NaN,NaN
2,18,2008.0,9.0,576.0,15.68,12.76,3.79,82.0,73.0,8.3,709.0,7.0
3,18,2009.0,2.0,576.0,15.61,10.93,3.28,80.0,73.0,8,192.0,7.0
4,22,1996.0,11.0,576.0,11.27,8.89,10.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16714,15,2016.0,0.0,293.0,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16715,0,2006.0,2.0,258.0,0.00,0.01,0.00,NaN,NaN,NaN,NaN,NaN
16716,10,2016.0,3.0,259.0,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN
16717,2,2003.0,7.0,143.0,0.01,0.00,0.00,NaN,NaN,NaN,NaN,NaN


In [178]:
base = base[base['NA_Sales'] > 0]
base = base[base['EU_Sales'] > 0]
base = base[base['JP_Sales'] > 0]

In [179]:
base.describe()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Count,Rating
count,2420.000000,2407.000000,2420.000000,2419.000000,2420.000000,2420.000000,2420.000000,1738.000000,1738.000000,1790.000000,1788.000000
mean,15.783884,2006.600332,5.260331,298.319967,0.890103,0.563583,0.304215,75.861335,41.989643,315.494972,3.743848
std,9.383386,6.648943,4.310499,185.779504,1.884643,1.158124,0.668082,11.512592,22.138126,746.867538,3.093460
min,0.000000,1983.000000,0.000000,0.000000,0.010000,0.010000,0.010000,33.000000,4.000000,4.000000,0.000000
25%,10.000000,2002.500000,1.000000,172.000000,0.130000,0.060000,0.030000,69.000000,25.000000,34.000000,0.000000
50%,18.000000,2008.000000,5.000000,293.000000,0.360000,0.210000,0.090000,77.000000,39.000000,83.000000,6.000000
75%,23.000000,2012.000000,10.000000,488.000000,0.930000,0.600000,0.280000,84.000000,57.000000,242.000000,7.000000
max,30.000000,2016.000000,11.000000,576.000000,41.360000,28.960000,10.220000,98.000000,113.000000,10179.000000,7.000000


In [180]:
### Aqui colocar o removedor de outliers. Criar biblioteca e por na engenharia.

In [181]:
import pandas as  pd
import numpy as np
from fancyimpute import IterativeImputer

In [182]:
def identify_outliers(data, column):
    q1 = np.percentile(data[column],25)
    q3 = np.percentile(data[column],75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5*iqr
    upper_bound = q3 + 1.5*iqr
    for i in range(0,len(data)):
        if data.loc[i,column] < lower_bound or data.loc[i,column] > upper_bound:
            data.loc[i,column] = np.NAN
    return data


def replace_outliers(data,imputer):
    for d in data.columns:
        data = identify_outliers(data,d)
    data_imputed = imputer.fit_transform(data)
    data = pd.DataFrame(data_imputed,columns = data.columns)
    return data


def drop_outliers(data):
    for d in data.columns:
        data = identify_outliers(data,d)
    data = data.dropna()
    return data

def cleaning_dataset(dataset):
    data_imputed = imputer.fit_transform(dataset)
    dataset = pd.DataFrame(data_imputed,columns = dataset.columns)
    for i in range(0,10):
        dataset = replace_outliers(dataset, imputer)
    dataset = drop_outliers(dataset)
    return dataset

In [183]:
base.isna().sum()

Platform             0
Year_of_Release     13
Genre                0
Publisher            1
NA_Sales             0
EU_Sales             0
JP_Sales             0
Critic_Score       682
Critic_Count       682
User_Score         630
User_Count         630
Rating             632
dtype: int64

In [184]:
base = cleaning_dataset(base)

In [185]:
base

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
0,18.0,2006.000000,2.0,576.0,1.039475,0.639812,0.178415,76.000000,51.000000,8.000000,322.000000,7.000000
1,4.0,2008.185187,7.0,576.0,0.742448,0.435876,0.131503,79.068472,49.464569,7.743135,199.533907,6.672206
2,18.0,2008.000000,9.0,576.0,1.012770,0.600931,0.179835,82.000000,73.000000,8.300000,318.928522,7.000000
3,18.0,2009.000000,2.0,576.0,0.836274,0.507670,0.150315,80.000000,73.000000,8.000000,192.000000,7.000000
4,22.0,1996.000000,11.0,576.0,1.896013,1.101736,0.287888,96.441040,57.162562,9.256040,530.521832,6.320071
...,...,...,...,...,...,...,...,...,...,...,...,...
2415,28.0,2007.000000,8.0,293.0,0.010000,0.010000,0.030000,79.000000,39.000000,7.900000,17.000000,2.000000
2416,18.0,2006.000000,6.0,177.0,0.030000,0.010000,0.010000,74.157819,32.000000,7.662810,8.000000,7.000000
2417,1.0,2007.000000,2.0,35.0,0.010000,0.010000,0.010000,68.000000,30.000000,6.300000,4.000000,7.000000
2418,20.0,2016.000000,0.0,177.0,0.010000,0.020000,0.010000,74.337860,43.728950,7.800000,11.000000,6.000000


In [186]:
base.describe()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
count,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000
mean,15.782141,2006.887808,5.258371,298.306153,0.530578,0.322586,0.110342,77.110673,38.786242,7.827762,127.312874,3.745523
std,9.384935,5.995188,4.310312,185.779059,0.481364,0.309343,0.093392,8.475429,19.616166,0.913438,190.493274,2.683982
min,0.000000,1990.000000,0.000000,0.000000,0.010000,0.010000,0.010000,56.000000,4.000000,5.300000,-332.795160,0.000000
25%,10.000000,2003.000000,1.000000,172.000000,0.130000,0.060000,0.030000,72.000000,22.997142,7.200000,22.000000,0.000000
50%,18.000000,2008.000000,5.000000,293.000000,0.360000,0.210000,0.080000,77.351765,37.000000,7.900000,78.000000,4.003854
75%,23.000000,2012.000000,10.000000,488.000000,0.840000,0.520000,0.170000,83.000000,51.000000,8.500000,259.000000,6.000000
max,30.000000,2016.000000,11.000000,576.000000,1.900000,1.210000,0.380000,98.000000,93.000000,10.436146,614.473624,8.027770


In [187]:
base = base[base['User_Count'] > 0]
base.describe()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Rating
count,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000,2100.000000
mean,15.045714,2008.443994,5.314286,289.222183,0.529316,0.335983,0.099755,76.749623,41.550435,7.673134,171.424742,3.771515
std,9.219560,4.659380,4.399671,183.655329,0.482362,0.316789,0.090052,8.982177,19.521767,0.864503,160.885918,2.865788
min,0.000000,1990.000000,0.000000,0.000000,0.010000,0.010000,0.010000,56.000000,4.000000,5.300000,0.482564,0.000000
25%,10.000000,2006.000000,0.750000,125.000000,0.130000,0.060000,0.020000,71.000000,27.000000,7.100000,38.000000,0.000000
50%,15.000000,2009.000000,5.000000,258.000000,0.350000,0.220000,0.070000,76.275890,41.000000,7.700000,104.000000,4.442623
75%,21.000000,2012.000000,10.000000,422.000000,0.835373,0.548656,0.151663,83.000000,54.000000,8.300000,295.248656,6.000000
max,30.000000,2016.000000,11.000000,576.000000,1.900000,1.210000,0.380000,98.000000,93.000000,10.436146,614.473624,8.027770


In [188]:
base.to_csv('base2.csv', index = False)